In [1]:
%load_ext autoreload
%autoreload 2
from dataset import iterable_dataset
from models.music_generation_gru import MusicGRU
import os
from utils import training_rnn, data_reading
import torch
from torch.utils.data import DataLoader
import pickle
import matplotlib.pyplot as plt
from utils import data_reading, music_generation
from models import music_generation_VAE
from dataset import iterable_dataset

In [2]:
FILES_TO_READ = 25000
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
PROJECT_DIR ="D:/522 project"
PPR_DIR = "lpd_5_pianoroll"
SEQ_LEN = 48
RESOLUTION = 12
TRAIN_SPLIT = 0.8
HIDDEN_SIZE = 512
BATCH_SIZE = 32
N_LAYERS = 6
ALPHA = 5
LR = 0.0001
LR_LAMBDA = 1
EPOCH = 20
AUTOSAVE = True
BUFFER_SIZE = 1000
REQUIRED_INSTRUMENTS = ["Guitar", "Piano"]
PATH = os.path.join(PROJECT_DIR, PPR_DIR)
GUITAR_MODEL_STATE_PATH = "./saved_model_and_data/vae/vae_guitar/G_vae_trained_r12s48d27e20"
PIANO_MODEL_STATE_PATH = "./saved_model_and_data/vae/vae_piano/P_vae_trained_r12s48d25e20"
GUITAR_FEATURES_SAVE_PATH = "./saved_model_and_data/transformed_features/guitar"

In [3]:
all_files_list = data_reading.read_all_npzs(PATH, shuffle=True)
guitar_piano_dataset = iterable_dataset.IterableSongDataset(all_files_list, SEQ_LEN, RESOLUTION, REQUIRED_INSTRUMENTS, 
                                          method="one_step_window", normalize=False, binarize=False,
                                          buffer_size=BUFFER_SIZE, files_to_read=FILES_TO_READ)
guitar_piano_loader = DataLoader(guitar_piano_dataset, batch_size = BATCH_SIZE, drop_last=True)

In [4]:
piano_model = music_generation_VAE.ConvVAE(["Piano"], HIDDEN_SIZE)
piano_model.load_state_dict(torch.load(PIANO_MODEL_STATE_PATH))
piano_model.to(DEVICE)

ConvVAE(
  (enc_conv1): Conv2d(1, 64, kernel_size=(4, 4), stride=(2, 2))
  (enc_conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2))
  (enc_conv3): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2))
  (enc_conv4): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2))
  (trans_enc_lin1): Linear(in_features=7168, out_features=512, bias=True)
  (trans_enc_lin2): Linear(in_features=512, out_features=256, bias=True)
  (trans_enc_lin3): Linear(in_features=256, out_features=513, bias=True)
  (trans_dec1): Linear(in_features=512, out_features=256, bias=True)
  (trans_dec2): Linear(in_features=256, out_features=512, bias=True)
  (trans_dec3): Linear(in_features=512, out_features=7168, bias=True)
  (dec_conv1): ConvTranspose2d(512, 256, kernel_size=(3, 3), stride=(2, 2))
  (dec_conv2): ConvTranspose2d(256, 128, kernel_size=(3, 3), stride=(2, 2))
  (dec_conv3): ConvTranspose2d(128, 64, kernel_size=(3, 3), stride=(2, 2))
  (dec_conv4): ConvTranspose2d(64, 1, kernel_size=(4, 4), stride=(2, 2))

In [5]:
guitar_model = music_generation_VAE.ConvVAE(["Guitar"], HIDDEN_SIZE)
guitar_model.load_state_dict(torch.load(GUITAR_MODEL_STATE_PATH))
guitar_model.to(DEVICE)

ConvVAE(
  (enc_conv1): Conv2d(1, 64, kernel_size=(4, 4), stride=(2, 2))
  (enc_conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2))
  (enc_conv3): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2))
  (enc_conv4): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2))
  (trans_enc_lin1): Linear(in_features=7168, out_features=512, bias=True)
  (trans_enc_lin2): Linear(in_features=512, out_features=256, bias=True)
  (trans_enc_lin3): Linear(in_features=256, out_features=513, bias=True)
  (trans_dec1): Linear(in_features=512, out_features=256, bias=True)
  (trans_dec2): Linear(in_features=256, out_features=512, bias=True)
  (trans_dec3): Linear(in_features=512, out_features=7168, bias=True)
  (dec_conv1): ConvTranspose2d(512, 256, kernel_size=(3, 3), stride=(2, 2))
  (dec_conv2): ConvTranspose2d(256, 128, kernel_size=(3, 3), stride=(2, 2))
  (dec_conv3): ConvTranspose2d(128, 64, kernel_size=(3, 3), stride=(2, 2))
  (dec_conv4): ConvTranspose2d(64, 1, kernel_size=(4, 4), stride=(2, 2))

In [6]:
import numpy as np
guitar_model.eval()
piano_model.eval()
i=1
x=0
import csv
res_list = []
print(guitar_piano_dataset.get_track_order())
for song_seq, _ in guitar_piano_dataset:
    song_seq = torch.from_numpy(song_seq).unsqueeze(0).float().to(DEVICE)
    piano_seq = song_seq[:, 0:1, :, :]
    guitar_seq = song_seq[:, 1:2, :, :]
    guitar_transformed_track = guitar_model.encode(guitar_seq).squeeze().detach().cpu().numpy()
    piano_transformed_track = piano_model.encode(piano_seq).squeeze().detach().cpu().numpy()
    transformed_track = np.stack([guitar_transformed_track, piano_transformed_track])
    res_list.append(transformed_track)
    if i%10000 == 0:
        np.save(os.path.join(GUITAR_FEATURES_SAVE_PATH, f"guitar_with_piano_vae_features{x}"), np.stack(res_list))
        del res_list
        res_list = []
        x+=1
    i+=1
np.save(os.path.join(GUITAR_FEATURES_SAVE_PATH, f"guitar_with_piano_vae_features_final"), np.stack(res_list))

['Piano', 'Guitar']
reloading
reloading
reloading
reloading
reloading
reloading
reloading
reloading
reloading
reloading
reloading
reloading
reloading
reloading
reloading
reloading
reloading
reloading
reloading
reloading
reloading
reloading
